# Data processing - Qredo transactions

#### Maria Silva, June 2023

In [5]:
import os
import datetime
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

## QL2 transactions

### Read raw data

In [6]:
file1 = os.path.realpath("../data/billable_ql2_first_million_2023-05-25T15_02_18.15685Z.csv")
file2 = os.path.realpath("../data/billable_ql2_second_million_2023-05-25T15_02_24.562051Z.csv")

raw_qrdo_df = pd.concat([pd.read_csv(file1), pd.read_csv(file2)])
raw_qrdo_df["created_at"] = pd.to_datetime(raw_qrdo_df["to_timestamp"], utc=True)
raw_qrdo_df["day"] = pd.to_datetime(raw_qrdo_df["created_at"].dt.date)

raw_qrdo_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1163376 entries, 0 to 163375
Data columns (total 18 columns):
 #   Column             Non-Null Count    Dtype              
---  ------             --------------    -----              
 0   seq_index          1163376 non-null  int64              
 1   tx_id              1163376 non-null  object             
 2   billable_type_s    4608 non-null     object             
 3   billable_status_s  4608 non-null     object             
 4   tx_type            1163376 non-null  object             
 5   chain_tx_id        1163376 non-null  object             
 6   wallet_id          1163376 non-null  object             
 7   fund_id            1163376 non-null  object             
 8   asset              1163376 non-null  object             
 9   asset_scale        1163376 non-null  int64              
 10  amount             1163376 non-null  int64              
 11  fees               1163376 non-null  int64              
 12  net_amount         1

In [7]:
len(raw_qrdo_df[raw_qrdo_df["amount"] <0])

8

In [8]:
raw_qrdo_df.groupby("benefit_of")["tx_id"].nunique().sort_values(ascending=False)

benefit_of
Staking event           370770
Vesting event           155441
QredoStation              1236
PowerSwap                  272
1                          170
                         ...  
L                            1
Ledger                       1
MGL UK                       1
MKG Enterprises Corp         1
you                          1
Name: tx_id, Length: 312, dtype: int64

### Clean raw data

Some notes here:

* Some transferIn + transferOut  relate to staking and vesting events, which are filtered using the "benefit_of" field. We will give these transactions their own transaction type
* The other transferIn + transferOut are layer 2 transactions. For each transferIn, there is an equivalent transferOut. We will consider the transferOut only and rename these transactions as QL2.
* swapIn and swapOut will be excluded (there is only 4 of these transactions)

In [9]:
filter_df = raw_qrdo_df[raw_qrdo_df["amount"]>0].copy()

staking_df = filter_df[(filter_df["benefit_of"] == "Staking event") & (filter_df["tx_type"] == "transferIn")].copy()
staking_df["tx_type"] = "staking"

# Need to double check this!!!
vesting_df = filter_df[filter_df["benefit_of"] == "Vesting event"].copy()
vesting_df["tx_type"] = np.where(vesting_df["tx_type"]=="transferIn", "vestingIn", "vestingOut")

ql2_internal_df = filter_df[(~filter_df["benefit_of"].isin(["Staking event", "Vesting event"])) & (filter_df["tx_type"] == "transferOut")].copy()
ql2_internal_df["tx_type"] = "ql2_internal"

others_df = filter_df[filter_df["tx_type"].isin(["deposit", "withdraw"])].copy()

In [23]:
ql2_df = pd.concat([staking_df, ql2_internal_df, vesting_df, others_df], ignore_index=True)

ql2_df["tx_group"] = np.where(ql2_df["asset"]=="QRDO", "ql2_QRDO", "ql2_others")
ql2_df["token"] = ql2_df["asset"]

ql2_df = ql2_df.sort_values("to_timestamp").reset_index(drop=True)
ql2_df = ql2_df[["created_at", "day","tx_group", "tx_type", "fund_id", "wallet_id", "token"]].copy()

ql2_df.head()

,created_at,day,tx_group,tx_type,fund_id,wallet_id,token
0,2020-09-01 16:51:08+00:00,2020-09-01,ql2_others,deposit,1guxjFdsfO91UwSpbKFFA5x8vDq,9WTr4Sycv3rP1QE3ekhHi7WBN1PQzHoP8wWsoXFnS7Fo,BTC-TESTNET
1,2020-09-01 16:51:10+00:00,2020-09-01,ql2_others,ql2_internal,1guxjFdsfO91UwSpbKFFA5x8vDq,9WTr4Sycv3rP1QE3ekhHi7WBN1PQzHoP8wWsoXFnS7Fo,BTC-TESTNET
2,2020-09-01 16:51:16+00:00,2020-09-01,ql2_others,withdraw,1guxjFrVzNsBzGapFqhpN2pjAX4,H6SkJMRh3paLXezaDRMKZaKWhYrgYLeHf1eWuKAByMMT,BTC-TESTNET
3,2020-09-01 18:33:02+00:00,2020-09-01,ql2_others,ql2_internal,1gv9UES92rUjT8DT1MO1mX7WVuK,4SApMHPxM4c489y4xNK4S9vyVWgqfDGbVXy5g9qwufuz,BTC-TESTNET
4,2020-09-01 18:33:05+00:00,2020-09-01,ql2_others,deposit,1gv9UES92rUjT8DT1MO1mX7WVuK,4SApMHPxM4c489y4xNK4S9vyVWgqfDGbVXy5g9qwufuz,BTC-TESTNET


### Compute counts

In [15]:
agg_ql2_df = ql2_df.groupby(["day","tx_group", "tx_type"]).size().reset_index()
agg_ql2_df.columns = ['day', 'tx_group', 'tx_type', 'tx_count']
agg_ql2_df.head()

,day,tx_group,tx_type,tx_count
0,2020-09-01,ql2_others,deposit,3
1,2020-09-01,ql2_others,ql2_internal,3
2,2020-09-01,ql2_others,withdraw,3
3,2020-09-02,ql2_others,deposit,3
4,2020-09-02,ql2_others,ql2_internal,8


## MMI transactions

### Read raw data

In [19]:
file = os.path.realpath("../data/billable_events_mmi_2023-05-26T10_38_15.183589Z.csv")

raw_mmi_df = pd.read_csv(file) 
raw_mmi_df["created_at"] = pd.to_datetime(raw_mmi_df["timestamp"], unit="s", utc=True)
raw_mmi_df["day"] = pd.to_datetime(raw_mmi_df["created_at"].dt.date)
raw_mmi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105028 entries, 0 to 105027
Data columns (total 40 columns):
 #   Column               Non-Null Count   Dtype              
---  ------               --------------   -----              
 0   tx_id                105028 non-null  object             
 1   source_tx            60275 non-null   object             
 2   billable_type_s      60275 non-null   object             
 3   billable_status_s    60275 non-null   object             
 4   chain_name           94103 non-null   object             
 5   tx_type              69449 non-null   object             
 6   tx_id.1              105028 non-null  object             
 7   wallet_id            105028 non-null  object             
 8   fund_id              105028 non-null  object             
 9   chain_id             104925 non-null  float64            
 10  tx_hash              105027 non-null  object             
 11  timestamp            105028 non-null  int64              
 12  ex

In [26]:
raw_mmi_df.head(1).T

,0
tx_id,2QJaB5hAfNixaEPhDS3wAR27ukp
source_tx,NaN
billable_type_s,NaN
billable_status_s,NaN
chain_name,NaN
tx_type,send
tx_id.1,2QJaB5hAfNixaEPhDS3wAR27ukp
wallet_id,4458gw1TBZqEpwP9xPy73QXxW3ZGpqDsbWCG7yJBWYBg
fund_id,29w0cQGk5MSVHyy7lFivL1dF4g1
chain_id,10.0


In [21]:
raw_mmi_df.groupby("tx_type").size().sort_values(ascending=False).head(20)

tx_type
send                        12445
deposit                      5285
empty                        4129
swap                         3241
swapExactTokensForTokens     3228
withdraw                     2936
multicall                    2146
repay                        2138
addLiquidity                 1728
mint                         1643
receive                      1505
execute                      1225
anySwapOutUnderlying         1180
simpleSwap                    874
borrow                        851
swapExactTokensForETH         842
addLiquidityETH               703
add_liquidity                 660
swapExactETHForTokens         638
outboundTransferTo            638
dtype: int64

### Clean raw data

In [27]:
mmi_df = raw_mmi_df.sort_values("created_at").reset_index(drop=True).copy()
mmi_df["tx_group"] = "mmi"
mmi_df["token"] = np.nan
mmi_df = mmi_df[["created_at", "day","tx_group", "tx_type", "fund_id", "wallet_id", "token"]].copy()

mmi_df.head()

,created_at,day,tx_group,tx_type,fund_id,wallet_id,token
0,2021-10-04 11:46:05+00:00,2021-10-04,mmi,NaN,1z2YMd3fbZtA3ydq5qkUAjpxQWv,Ekc3mg3D6VVQaywfDzZCLAv5iQhX73bSQMoE9MjKPUyd,NaN
1,2021-10-04 12:33:18+00:00,2021-10-04,mmi,NaN,1z2YMd3fbZtA3ydq5qkUAjpxQWv,Ekc3mg3D6VVQaywfDzZCLAv5iQhX73bSQMoE9MjKPUyd,NaN
2,2021-10-06 10:05:04+00:00,2021-10-06,mmi,NaN,1z2YMd3fbZtA3ydq5qkUAjpxQWv,Ekc3mg3D6VVQaywfDzZCLAv5iQhX73bSQMoE9MjKPUyd,NaN
3,2021-10-08 09:39:23+00:00,2021-10-08,mmi,NaN,1z89IOaPmuZeZJTyq1iPmhFV6gC,BpJgXJTkWv4ruLJaSxZChZmje3BiKwpy8jeVJxKMsbbh,NaN
4,2021-10-08 09:42:10+00:00,2021-10-08,mmi,NaN,1z89IOaPmuZeZJTyq1iPmhFV6gC,BpJgXJTkWv4ruLJaSxZChZmje3BiKwpy8jeVJxKMsbbh,NaN


### Compute counts

In [28]:
agg_mmi_df = mmi_df.groupby(["day","tx_group"]).size().reset_index()
agg_mmi_df.columns = ['day', 'tx_group','tx_count']
agg_mmi_df["tx_type"] = "mmi"
agg_mmi_df.head()

,day,tx_group,tx_count,tx_type
0,2021-10-04,mmi,2,mmi
1,2021-10-06,mmi,1,mmi
2,2021-10-08,mmi,9,mmi
3,2021-10-11,mmi,1,mmi
4,2021-10-13,mmi,3,mmi


## Internal transactions

### Read raw data

In [29]:
file = os.path.realpath("../data/covalent_parsed_transaction_data_2023-05-30T08_34_45.138349Z.csv")

raw_int_df = pd.read_csv(file) 
#raw_mmi_df["created_at"] = pd.to_datetime(raw_mmi_df["timestamp"], unit="s", utc=True)
#raw_mmi_df["day"] = pd.to_datetime(raw_mmi_df["created_at"].dt.date)
raw_int_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 592 entries, 0 to 591
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   chain_name          592 non-null    object 
 1   value               592 non-null    object 
 2   tx_hash             592 non-null    object 
 3   fees_paid           592 non-null    int64  
 4   gas_price           592 non-null    int64  
 5   gas_quote           545 non-null    float64
 6   gas_spent           592 non-null    int64  
 7   tx_offset           592 non-null    int64  
 8   sucessful           0 non-null      float64
 9   to_address          592 non-null    object 
 10  gas_offered         592 non-null    int64  
 11  value_quote         545 non-null    float64
 12  block_height        592 non-null    int64  
 13  from_address        592 non-null    object 
 14  gas_quote_rate      545 non-null    float64
 15  block_signed_at     592 non-null    object 
 16  pretty_g

In [30]:
raw_int_df

,chain_name,value,tx_hash,fees_paid,gas_price,gas_quote,gas_spent,tx_offset,sucessful,to_address,...,value_quote,block_height,from_address,gas_quote_rate,block_signed_at,pretty_gas_quote,to_address_label,from_address_label,pretty_value_quote,log_events
0,bsc-mainnet,0,0x7aa4673823407072a7e76d18e14cee50aef5e3b77009...,175519000000000,1000000000,0.056512,175519,115,NaN,0xb9b9987fa8db666402fed80263b220203de26791,...,0.0,27845441,0xd5687e563a78b86b52804fc8afb844da0e2afc81,321.972351,2023-05-02T01:57:18Z,$0.06,NaN,NaN,$0.00,"[{""decoded"": {""name"": ""Transfer"", ""params"": [{..."
1,bsc-mainnet,0,0xb8c27dd07f840a41ff11dca05b90450a8ed250a28eb5...,430747000000000,1000000000,0.145699,430747,108,NaN,0xc391a88165d9e23ebd28c2e220c5f9900d1d8f35,...,0.0,27813609,0x1d6c49705248a0ff4aa1d20ecdc0e1ec05f8eb06,338.247986,2023-04-30T23:24:08Z,$0.15,NaN,NaN,$0.00,"[{""decoded"": {""name"": ""Approval"", ""params"": [{..."
2,bsc-mainnet,0,0x6b315299fc07515811d55096c156903bbe02b4386677...,368957589000000,1001000000,0.124799,368589,93,NaN,0xc85f1823468a21a8f9ff0728615a037c81a176ac,...,0.0,27813609,0xa64d713502ec8ace41af84953ade60afe217e0c4,338.247986,2023-04-30T23:24:08Z,$0.12,NaN,NaN,$0.00,"[{""decoded"": {""name"": ""Approval"", ""params"": [{..."
3,bsc-mainnet,0,0x0c155e9fefa742e51af4482acc9a93aa52c788881623...,108453000000000,3000000000,0.036684,36151,42,NaN,0x55d398326f99059ff775485246999027b3197955,...,0.0,27813601,0xd275e5cb559d6dc236a5f8002a5f0b4c8e610701,338.247986,2023-04-30T23:23:44Z,$0.04,Binance-Peg BSC-USD (BSC-USD),NaN,$0.00,"[{""decoded"": {""name"": ""Transfer"", ""params"": [{..."
4,bsc-mainnet,0,0xc36e77c4ec8c8f425c3c28c6704ec09fd6784d148527...,450695000000000,1000000000,0.152447,450695,129,NaN,0xc391a88165d9e23ebd28c2e220c5f9900d1d8f35,...,0.0,27813479,0x95d4f29f2c155218ebe19120fe55e34e1bae7f1f,338.247986,2023-04-30T23:17:38Z,$0.15,NaN,NaN,$0.00,"[{""decoded"": {""name"": ""Approval"", ""params"": [{..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587,eth-goerli,0,0x64b3031ebb3157133f6aa9a3fd0a27c776cb282041b9...,38413002425160,1500039145,NaN,25608,13,NaN,0xd275e5cb559d6dc236a5f8002a5f0b4c8e610701,...,NaN,8455052,0x4f61a6ae3f1c2b4e99e394a8ffd7802362048064,NaN,2023-02-08T07:52:24Z,NaN,NaN,NaN,NaN,[]
588,eth-goerli,0,0xafd8be5ab063a41738f08f72b76dfcff4395312cb3d1...,45108001533672,1500000051,NaN,30072,27,NaN,0xd275e5cb559d6dc236a5f8002a5f0b4c8e610701,...,NaN,8443468,0x2ef3cd550e53b19eec48c12f8d27da84b3323fd5,NaN,2023-02-06T08:26:48Z,NaN,NaN,NaN,NaN,[]
589,eth-goerli,0,0x3af749796377d94b16f1ca16e3e43c90ca9056948e9b...,45108001533672,1500000051,NaN,30072,19,NaN,0xd275e5cb559d6dc236a5f8002a5f0b4c8e610701,...,NaN,8443468,0x2ef3cd550e53b19eec48c12f8d27da84b3323fd5,NaN,2023-02-06T08:26:48Z,NaN,NaN,NaN,NaN,[]
590,eth-goerli,0,0x7968be06b6b2ff8e7342b7a991d5012095c16c87f0c0...,45108001593816,1500000053,NaN,30072,33,NaN,0xd275e5cb559d6dc236a5f8002a5f0b4c8e610701,...,NaN,8443465,0x2ef3cd550e53b19eec48c12f8d27da84b3323fd5,NaN,2023-02-06T08:26:00Z,NaN,NaN,NaN,NaN,[]


### Clean raw data

### Compute counts

## Join all data

In [ ]:
tx_df = pd.concat([ql2_df, mmi_df], ignore_index=True)
tx_df = tx_df.sort_values("created_at").reset_index(drop=True)

file = os.path.realpath("../data/clean_tx.csv")
tx_df.to_csv(file, index=False)

tx_df.head()

In [ ]:
agg_df = pd.concat([agg_ql2_df, agg_mmi_df], ignore_index=True)
agg_df = agg_df.sort_values("day").reset_index(drop=True)

file = os.path.realpath("../data/agg_tx.csv")
agg_df.to_csv(file, index=False)

agg_df.head()

In [ ]:
agg_df.groupby("tx_type")["tx_count"].sum().sort_values(ascending=False)